NOTE: this wont run properly if run with llama_index versions lower than 11 (becaus of triplet parser function)

In [1]:
#!pip install llama-index-llms-ollama

In [20]:
from groq import Groq
from llama_index.core import Settings, ServiceContext, StorageContext, SimpleDirectoryReader
from llama_index.llms.groq import Groq as Groq_llamaindex
from llama_index.llms.ollama import Ollama as Ollama_llamaindex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
#from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex
from llama_index.graph_stores.neo4j import Neo4jGraphStore, Neo4jPropertyGraphStore
from milvus import default_server
from dotenv import load_dotenv
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor, SimpleLLMPathExtractor, ImplicitPathExtractor, DynamicLLMPathExtractor

from llama_index.core.node_parser import (
    SentenceSplitter,
    SemanticSplitterNodeParser,
)



load_dotenv()
import json
import os
import numpy as np


# Retrieve API keys and credentials securely
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_URL = os.getenv('NEO4J_URL', 'bolt://localhost:7687')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE', 'neo4j')


client = Groq(api_key = GROQ_API_KEY)

llm = Groq_llamaindex(model="llama3-groq-70b-8192-tool-use-preview",
                       api_key='your api key', #using data uploader for now
                       temperature=0)

#llm = Ollama_llamaindex(model='mistral:instruct', temperature=0)

Settings.llm = llm
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

StorageContext.llm = llm
ServiceContext.llm = llm

In [3]:
#llm.complete('write a poem')

We now load the files with document readers. we select the following
- 25 simple files from nzta
- 15 annexed files from  nzta
- 15 table files from nzta
- 15 table annexed files from nzta
- 25 files from mot

In [4]:
username = NEO4J_USERNAME
password =  NEO4J_PASSWORD
url = NEO4J_URL
database = NEO4J_DATABASE



In [8]:
password

'nzta_neo4j'

In [9]:

property_graph_store = Neo4jPropertyGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)
storage_context = StorageContext.from_defaults(property_graph_store=property_graph_store)



Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


In [6]:
# print working dir

print(os.getcwd())

/Users/felipenavarro/Documents/Auckland/nzta/nzta_dev/NZTA-GraphRAG


In [7]:
os.path.exists("/Users/felipenavarro/Documents/Auckland/nzta/nzta_dev/NZTA-GraphRAG/processed_files_test.txt")

True

In [21]:
import os
import shutil
import time
from llama_index.core import PropertyGraphIndex
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor, SimpleLLMPathExtractor, ImplicitPathExtractor
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader
import nest_asyncio
nest_asyncio.apply()


# Initialize directories, batch size, wait time, and skip count

source_dir = "transport_data/sample_files/nzta/simple_files_set_2"
target_dir = "transport_data/sample_files/nzta/temp_files"
batch_size = 2
wait_minutes = 1  # Specify the wait time in minutes
skip_files = 0  # Number of files to skip

# Create the target directory if it does not exist
if not os.path.exists(target_dir):
    os.makedirs(target_dir)
    

# Start by deleting anything under temp_files but not the directory itself
if os.path.exists(target_dir):
    for file_name in os.listdir(target_dir):
        file_path = os.path.join(target_dir, file_name)
        if os.path.isfile(file_path):
            os.remove(file_path)
        else:
            shutil.rmtree(file_path)

# Read the list of already processed files
processed_files = set()
if os.path.exists("/Users/felipenavarro/Documents/Auckland/nzta/nzta_dev/NZTA-GraphRAG/processed_files_test.txt"):
    print("Processed files exist")
    with open("/Users/felipenavarro/Documents/Auckland/nzta/nzta_dev/NZTA-GraphRAG/processed_files_test.txt", "r") as f:
        processed_files = set(line.strip() for line in f)

# List all files in the source directory
full_file_list = os.listdir(source_dir)
print(f"Processed files are: {processed_files}")
# Filter out files that have already been processed
full_file_list = [file for file in full_file_list if file not in processed_files]

# Optionally, skip the first 'k' files
if skip_files > 0:
    full_file_list = full_file_list[skip_files:]

# Calculate the number of batches needed
num_batches = len(full_file_list) // batch_size + (1 if len(full_file_list) % batch_size > 0 else 0)

for i in range(num_batches):
    # Determine the start and end indices for the current batch
    start_idx = i * batch_size
    end_idx = min(start_idx + batch_size, len(full_file_list))

    # Get the current batch of files
    current_batch = full_file_list[start_idx:end_idx]
    print(f"Processing batch {i+1}/{num_batches}: {current_batch}")

    # Copy the current batch of files to the target directory
    for file_name in current_batch:
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(target_dir, file_name))

    # Process the current batch of files
    graph_documents = SimpleDirectoryReader(target_dir).load_data()


######## code below is for semantic splitter
#     # Initialize the semantic splitter
#     splitter = SemanticSplitterNodeParser(
#     buffer_size=1, breakpoint_percentile_threshold=95, embed_model=Settings.embed_model
#         )

# # Use the semantic splitter to get the nodes (chunks) from the documents
#     nodes = splitter.get_nodes_from_documents(graph_documents)
######## code above is for semantic splitter

    Settings.chunk_size = 512
    Settings.chunk_overlap = 20


# Original prompt modified by me

#     extract_prompt = (
#     "Extract up to {max_knowledge_triplets} knowledge triplets from the given text. "
#     "Each triplet should be in the form of (head, relation, tail) with their respective types.\n"
#     "---------------------\n"
#     "INITIAL ONTOLOGY:\n"
#     "Entity Types: {allowed_entity_types}\n"
#     "Relation Types: {allowed_relation_types}\n"
#     "\n"
#     "Use these types as a starting point, but introduce new types if necessary based on the context.\n"
    
#     "For any reference to the New Zealand Transport Agency Waka Kotahi , including variations such as 'NZ Transport Agency Waka Kotahi',  ‘NZ Transport Agency,’ ‘National Transport Agency,’ ‘Waka Kotahi,’ or any similar phrasing, create the entity node as ‘NZTA.’ Ensure that all variations of this entity are consistently mapped to ‘NZTA’ in the output.\n"
#     "\n"
#     "GUIDELINES:\n"
#     "- Output in JSON format: [{{'head': '', 'head_type': '', 'relation': '', 'tail': '', 'tail_type': ''}}]\n"
#     "- Use the most complete form for entities (e.g., 'United States of America' instead of 'USA')\n"
#     "- Keep entities concise (3-5 words max)\n"
#     "- Break down complex phrases into multiple triplets\n"
#     "- Ensure the knowledge graph is coherent and easily understandable\n"
#     "---------------------\n"
#     "EXAMPLE:\n"
#     "Text: Tim Cook, CEO of Apple Inc., announced the new Apple Watch that monitors heart health. "
#     "UC Berkeley researchers studied the benefits of apples.\n"
#     "Output:\n"
#     "[{{'head': 'Tim Cook', 'head_type': 'PERSON', 'relation': 'CEO_OF', 'tail': 'Apple Inc.', 'tail_type': 'COMPANY'}},\n"
#     " {{'head': 'Apple Inc.', 'head_type': 'COMPANY', 'relation': 'PRODUCES', 'tail': 'Apple Watch', 'tail_type': 'PRODUCT'}},\n"
#     " {{'head': 'Apple Watch', 'head_type': 'PRODUCT', 'relation': 'MONITORS', 'tail': 'heart health', 'tail_type': 'HEALTH_METRIC'}},\n"
#     " {{'head': 'UC Berkeley', 'head_type': 'UNIVERSITY', 'relation': 'STUDIES', 'tail': 'benefits of apples', 'tail_type': 'RESEARCH_TOPIC'}}]\n"
#     "---------------------\n"
#     "Text: {text}\n"
#     "Output:\n"
# )

# # Prompt by chatgpt. 
   # NZTA-oriented Prompt
    extract_prompt = (
    "Extract up to {max_knowledge_triplets} knowledge triplets from the given text. "
    "Each triplet should be in the form of (head, relation, tail) with their respective types and properties.\n"
    "---------------------\n"
    "INITIAL ONTOLOGY:\n"
    "Entity Types: {allowed_entity_types}\n"
    "Entity Properties: {allowed_entity_properties}\n"
    "Relation Types: {allowed_relation_types}\n"
    "Relation Properties: {allowed_relation_properties}\n"
    "\n"
    "Use these types as a starting point, but introduce new types if necessary based on the context.\n"
    "\n"
    "GUIDELINES:\n"
    "- Output in JSON format: [{{'head': '', 'head_type': '', 'head_props': {{...}}, 'relation': '', 'relation_props': {{...}}, 'tail': '', 'tail_type': '', 'tail_props': {{...}}}}]\n"
    "- Use the most complete form for entities (e.g., 'New Zealand Transport Agency' instead of abbreviations)\n"
    "For any reference to the New Zealand Transport Agency, including variations such as 'NZ Transport Agency Waka Kotahi', ‘NZ Transport Agency,’ ‘National Transport Agency,’ ‘Waka Kotahi,’ or any similar phrasing, create the entity node as 'New Zealand Transport Agency.' Ensure all variations of this entity are consistently mapped to 'New Zealand Transport Agency' in the output.\n"
    "- Do not create entities like 'request' or 'response' as they appear frequently and do not contribute to meaningful graph structure.\n"
    "- Do not create entities that are numbers, prices, or non-essential terms (e.g., dates unless critical to the meaning).\n"
    "- Keep entities concise (3-5 words max).\n"
    "- Break down complex phrases into multiple triplets for better granularity.\n"
    "- Ensure the knowledge graph is coherent and easily understandable.\n"
    "- Ensure that all property values are primitive types (e.g., String, Integer, Float, Boolean) or arrays of these types. Do not use maps or other complex structures.\n"
    "---------------------\n"
    "EXAMPLE:\n"
    "Text: The New Zealand Transport Agency approved the funding for road maintenance on State Highway 1.\n"
    "Wellington City Council is collaborating with the New Zealand Transport Agency on improving road safety initiatives.\n"
    "Output:\n"
    "[{{'head': 'New Zealand Transport Agency', 'head_type': 'AGENCY', 'head_props': {{'prop1': 'val', ...}}, "
    "'relation': 'APPROVES', 'relation_props': {{'prop1': 'val', ...}}, "
    "'tail': 'funding for road maintenance', 'tail_type': 'FUNDING_DECISION', 'tail_props': {{'prop1': 'val', ...}}}},\n"
    " {{'head': 'Wellington City Council', 'head_type': 'ORGANIZATION', 'head_props': {{'prop1': 'val', ...}}, "
    "'relation': 'COLLABORATES_WITH', 'relation_props': {{'prop1': 'val', ...}}, "
    "'tail': 'New Zealand Transport Agency', 'tail_type': 'AGENCY', 'tail_props': {{'prop1': 'val', ...}}}},\n"
    " {{'head': 'road safety initiatives', 'head_type': 'PROJECT', 'head_props': {{'prop1': 'val', ...}}, "
    "'relation': 'IMPROVES', 'relation_props': {{'prop1': 'val', ...}}, "
    "'tail': 'State Highway 1', 'tail_type': 'INFRASTRUCTURE', 'tail_props': {{'prop1': 'val', ...}}}}]\n"
    "---------------------\n"
    "Text: {text}\n"
    "Output:\n"
)
    
    kg_extractor = DynamicLLMPathExtractor(
    llm=llm,
    extract_prompt=extract_prompt,
    max_triplets_per_chunk=15,
    
    num_workers=4,
    # Let the LLM infer entities and their labels (types) on the fly
    allowed_entity_types=['People', 'Organizations', 'Locations', 'Processes', 'Agreements', 'Buildings', 'Streets', 'Projects', 'Events','Policies', 'Rules', 'Laws'],
    # Let the LLM infer relationships on the fly
    allowed_relation_types=None,
    # LLM will generate any entity properties, set `None` to skip property generation (will be faster without)
    allowed_relation_props=[],
    # LLM will generate any relation properties, set `None` to skip property generation (will be faster without)
    allowed_entity_props=[],
    )

    graph_index = PropertyGraphIndex.from_documents(
    graph_documents,
    property_graph_store=property_graph_store,
    storage_context=storage_context,
    #kg_extractors=[SimpleLLMPathExtractor(llm=llm), ImplicitPathExtractor()],
    kg_extractors=[kg_extractor],
    embed_kg_nodes=True,
    show_progress=True)
     
    # Use the semantic splitter to get the nodes (chunks) from the documents
#     graph_index = PropertyGraphIndex(
#     nodes,  # Use the split nodes here
#     property_graph_store=property_graph_store,
#     storage_context=storage_context,
#     kg_extractors=[kg_extractor],
#     embed_kg_nodes=True,
#     show_progress=True
# )


    # Record the processed files immediately after processing
    with open("processed_files_test.txt", "a") as f:
        for file_name in current_batch:
            f.write(f"{file_name}\n")
            f.flush()  # Ensure data is written to disk
    print('Recorded processed files in processed_files.txt')

    # Wait for the specified number of minutes
    time.sleep(wait_minutes * 60)

    # Delete the processed files from the target directory
    for file_name in current_batch:
        os.remove(os.path.join(target_dir, file_name))

    print(f"Batch {i+1}/{num_batches} processed and cleaned up.\n")

print("All files processed. Processed files list updated in 'processed_files.txt'.")



Processed files exist
Processed files are: {'', 'oia-12115-response-letter.pdf', 'oia-6436-jordan-bond.pdf', 'oia-12662-response-letter.pdf', 'oia-13903-response-letter.pdf', 'oia-9406-response.pdf', 'oia-3496-michael-hayward-Item-1.pdf', 'oia-13118-response-letter.pdf', 'oia-0915-forbes.pdf', 'oia-5421-Robert-Pines.pdf', 'oia-12074-response-letter.pdf', 'oia-15834-response-letter.pdf', 'oia-9870-response.pdf', 'oia-9368-response.pdf', 'oia-5672-Phil-Pennington.pdf', 'oia-10465-george-block.pdf', 'OIA-1153-fuel-excise-refunds.pdf', 'oia-11764-response-letter.pdf', 'oia-12363-response-letter.pdf', 'OIA-13136-response-letter.pdf', 'oia-12236-response-letter.pdf', 'oia-5783-Jordan-Bond.pdf', 'oia-4379-Phil-Pennington.pdf', 'oia-9157-response.pdf', 'oia-3780-debrin-foxcroft-20180525.pdf', 'oia-15470-response-letter.pdf', 'oia-1129-keijser.pdf', 'oia-11388-response.pdf', 'oia-8452-response.pdf', 'oia-2940-Jo-Lines-MacKenzie.pdf', 'oia-4903-Andrew-Mcfarlane.pdf', 'oia-3727-bernard-orsman-201

Parsing nodes: 100%|██████████| 4/4 [00:00<00:00, 1809.84it/s]
Extracting and inferring knowledge graph from text: 100%|██████████| 5/5 [00:07<00:00,  1.50s/it]


changes are being registered
this is a node beofre poping the KG_NODES_KEY nodes and KG_RELATIONS_KEY relations
this is the node part
this is the relation part
this is kg_node 0 Conan Young ({'page_label': '1', 'file_name': 'oia-1243-conan-young.pdf', 'file_path': '/Users/felipenavarro/Documents/Auckland/nzta/nzta_dev/NZTA-GraphRAG/transport_data/sample_files/nzta/temp_files/oia-1243-conan-young.pdf', 'file_type': 'application/pdf', 'file_size': 442225, 'creation_date': '2024-10-16', 'last_modified_date': '2024-10-16', 'triplet_source_id': '9d43909c-d1ec-4540-acb0-620cde4e63e7'})
this is kg_rel 0 {'label': 'REQUESTED', 'source_id': 'Conan Young', 'target_id': 'Information under the Official Information Act 1982', 'properties': {'page_label': '1', 'file_name': 'oia-1243-conan-young.pdf', 'file_path': '/Users/felipenavarro/Documents/Auckland/nzta/nzta_dev/NZTA-GraphRAG/transport_data/sample_files/nzta/temp_files/oia-1243-conan-young.pdf', 'file_type': 'application/pdf', 'file_size': 4422

Generating embeddings: 100%|██████████| 11/11 [00:01<00:00,  5.94it/s]


This is an example of a kg relation to be embedded
Conan Young REQUESTED Information under the Official Information Act 1982 {}


...

This is an example of a kg relation to be embedded
file_path CONTAINS oia-11331-response.pdf {}


Generating embeddings: 100%|██████████| 7/7 [00:01<00:00,  5.63it/s]


I have succesuflly created relation embeddings


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The procedure has a deprecated field. ('config' used by 'apoc.meta.graphSample' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL apoc.meta.graphSample() YIELD nodes, relationships RETURN nodes, [rel in relationships | {name:apoc.any.property(rel, 'type'), count: apoc.any.property(rel, 'count')}] AS relationships"


Recorded processed files in processed_files.txt
Batch 6/6 processed and cleaned up.

All files processed. Processed files list updated in 'processed_files.txt'.


In [18]:
kg_extractor.extract_prompt.template

"Extract up to {max_knowledge_triplets} knowledge triplets from the given text. Each triplet should be in the form of (head, relation, tail) with their respective types and properties.\n---------------------\nINITIAL ONTOLOGY:\nEntity Types: {allowed_entity_types}\nEntity Properties: {allowed_entity_properties}\nRelation Types: {allowed_relation_types}\nRelation Properties: {allowed_relation_properties}\n\nUse these types as a starting point, but introduce new types if necessary based on the context.\n\nGUIDELINES:\n- Output in JSON format: [{{'head': '', 'head_type': '', 'head_props': {{...}}, 'relation': '', 'relation_props': {{...}}, 'tail': '', 'tail_type': '', 'tail_props': {{...}}}}]\n- Use the most complete form for entities (e.g., 'United States of America' instead of 'USA')\nFor any reference to the New Zealand Transport Agency Waka Kotahi , including variations such as 'NZ Transport Agency Waka Kotahi',  ‘NZ Transport Agency,’ ‘National Transport Agency,’ ‘Waka Kotahi,’ or an

In [19]:
PropertyGraphIndex.from_documents?

Signature:
PropertyGraphIndex.from_documents(
    documents: Sequence[llama_index.core.schema.Document],
    storage_context: Optional[llama_index.core.storage.storage_context.StorageContext] = None,
    show_progress: bool = False,
    callback_manager: Optional[llama_index.core.callbacks.base.CallbackManager] = None,
    transformations: Optional[List[llama_index.core.schema.TransformComponent]] = None,
    **kwargs: Any,
) -> ~IndexType
Docstring:
Create index from documents.

Args:
    documents (Optional[Sequence[BaseDocument]]): List of documents to
        build the index from.
File:      ~/Documents/Auckland/nzta/nzta_dev/NZTA-GraphRAG/nzta_dev_env/lib/python3.11/site-packages/llama_index/core/indices/base.py
Type:      method

In [20]:
"Extract up to {max_knowledge_triplets} knowledge triplets from the given text." 
"Each triplet should be in the form of (head, relation, tail) with their respective types and properties.\n"
"---------------------\nINITIAL ONTOLOGY:\nEntity Types: {allowed_entity_types}\nEntity Properties: {allowed_entity_properties}\nRelation 
"Types: {allowed_relation_types}\nRelation Properties: {allowed_relation_properties}\n\nUse these types as a starting point, "
"but introduce new types if necessary based on the context.\n\nGUIDELINES:\n-"
"Output in JSON format: [{{'head': '', 'head_type': '', 'head_props': {{...}}, 'relation': '', 'relation_props': {{...}}, 'tail': '', 'tail_type': '', 'tail_props': {{...}}}}]\n- "
"Use the most complete form for entities (e.g., 'United States of America' instead of 'USA')\n- Keep entities concise " 
"(3-5 words max)\n- Break down complex phrases into multiple triplets\n- Ensure the knowledge graph is coherent and easily "
"understandable\n---------------------\nEXAMPLE:\nText: Tim Cook, CEO of Apple Inc., announced the new Apple Watch that monitors heart health. UC Berkeley researchers studied the benefits of apples.\nOutput:\n[{{'head': 'Tim Cook', 'head_type': 'PERSON', 'head_props': {{'prop1': 'val', ...}}, 'relation': 'CEO_OF', 'relation_props': {{'prop1': 'val', ...}}, 'tail': 'Apple Inc.', 'tail_type': 'COMPANY', 'tail_props': {{'prop1': 'val', ...}}}},\n {{'head': 'Apple Inc.', 'head_type': 'COMPANY', 'head_props': {{'prop1': 'val', ...}}, 'relation': 'PRODUCES', 'relation_props': {{'prop1': 'val', ...}}, 'tail': 'Apple Watch', 'tail_type': 'PRODUCT', 'tail_props': {{'prop1': 'val', ...}}}},\n {{'head': 'Apple Watch', 'head_type': 'PRODUCT', 'head_props': {{'prop1': 'val', ...}}, 'relation': 'MONITORS', 'relation_props': {{'prop1': 'val', ...}}, 'tail': 'heart health', 'tail_type': 'HEALTH_METRIC', 'tail_props': {{'prop1': 'val', ...}}}},\n {{'head': 'UC Berkeley', 'head_type': 'UNIVERSITY', 'head_props': {{'prop1': 'val', ...}}, 'relation': 'STUDIES', 'relation_props': {{'prop1': 'val', ...}}, 'tail': 'benefits of apples', 'tail_type': 'RESEARCH_TOPIC', 'tail_props': {{'prop1': 'val', ...}}}}]\n---------------------\nText: {text}\nOutput:\n"

SyntaxError: unterminated string literal (detected at line 3) (678980661.py, line 3)